In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import re
from functools import reduce

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.sample(5)

In [ ]:
df.info()

# 1. Сколько мужчин было на корабле?

In [ ]:
len(df[df['Sex']=="male"])

In [ ]:
fig = px.pie(df, names='Sex')
fig.show()

# 2. Какая доля пассажиров (в %) выжила?

In [ ]:
print(f"{round(np.average(df['Survived'].values)*100,2)}%")

In [ ]:
fig = px.pie(df['Survived'].map({0:'Погибли',1:'Выжили'}), names='Survived')
fig.show()

#  3. Какая доля пассажиров (в %) от общего количества путешествовала во 2-ом классе? 

In [ ]:
print(f"{round(len(df[df['Pclass']==2])/len(df)*100,2)}%")

In [ ]:
fig = px.pie(df, names='Pclass')
fig.show()

# 4. Посчитайте среднее и медиану возраста всех людей на корабле.

In [ ]:
df_new = df.describe()[['Age']]
df_new

In [ ]:
df_new.loc[['50%','mean']]

# 5. Коррелируют ли число братьев/сестер с числом родителей/детей? 
Посчитайте корреляцию по Пирсону между признаками SibSp и 
Parch датасета.

In [ ]:
df_corr = df.corr(method='pearson')
df_corr

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr)
    )
)
fig.update_layout(
    autosize=False,
    width=700,
    height=700)

In [ ]:
df_corr.loc['Parch', 'SibSp']

In [ ]:
Parch=df.Parch-df.Parch.mean()
SibSp=df.SibSp-df.SibSp.mean()
(Parch*SibSp).mean()/df.Parch.std(ddof=0)/df.SibSp.std(ddof=0)

# 6. Какое самое популярное женское имя было на корабле? 
Извлеките из полного имени пассажира (колонка Name) его личное имя 
(FirstName). 

In [ ]:
df.query('Sex=="female"')[['Name']].sample(5)

In [ ]:
class Parser:
    met = 1
    __slots__ = ('dict')
    def __new__(cls,*args):
        if Parser.met:
            Parser.met = object.__new__(cls)
        return Parser.met
    
    def __init__(self):
        self.dict = {}
        
    @staticmethod
    def f(t):
        x = re.findall(r"[^(]*\((\w+)",t)# тут ищем имена в скобках Mrs. и Lady.
        if not x:
            x = re.findall(r"\. (\w+)", t)# тут проходимся по именам типо Miss. 
        return x[0]
        
    def __call__(self,df):
        for i in map(self.f,df.values):
            self.dict[i]=self.dict.get(i,0)+1
            
    def __str__(self):
        return str(reduce(lambda x,y: y if x[1]<y[1] else x,self.dict.items(), (None, 0)))


In [ ]:
%%time
pr = Parser()
pr(df.loc[df['Sex'] == 'female', 'Name'])

In [ ]:
from itertools import groupby
import operator
import random

In [1]:
%%time
class Subarray:
    __slots__ = ('temp', 'start', 'end', 'dist')
    
    def __init__(self, data):  # Тут data При инициализации это массив или любой итерируемые объект
        self.temp = map(lambda x: -1 if x&1 else 1, data) #O(n)
        self.start = 0
        self.end = 0
        
    def kadane(self):  #O(n) Алгооритм Kadane
        current = 0
        final = 0
        mp = -1
        for indx, i in enumerate(self.temp):
            current += i
            if current <= 0:
                mp = indx
                current = 0
            if final < current:
                final = current
                self.end = indx
                self.start = mp+1
            final = max(final, current)
        return final

    def __str__(self):
        dist = self.kadane()
        return str((self.start, self.end, dist))
    
    
with open("data/pi.txt", encoding='utf-8') as f:
    data = f.read()
data = map(int, data)
sb = Subarray(data)
print(sb)

(52, 387, 48)
CPU times: total: 0 ns
Wall time: 5 ms


In [ ]:
%%time


In [9]:
class Solution:

    def climbStairs(self, n: int) -> int:
        # n//2 - 2 шагов максимальное n - 1 шаг
        # k2 = n//2 to 0 
        def gen(n):
            for i in range(n):
                yield i+1
        def perm(k,t):
            gn = gen(k+t)
            po = 1
            for i in range(k):
                po*=next(gn)/(i+1)
            for i in range(t):
                po*=next(gn)/(i+1)
            return po
        k = n//2
        sumt = 0
        while k+1:
            sumt += perm(k,n-2*k)
            k-=1
        return int(sumt)

In [10]:
sl = Solution()
sl.climbStairs(3)

3

In [30]:
class Solution:
    def minimumTotal(self, triangle):
        n = len(triangle) - 1
        for i in reversed(range(n)):
            for q in range(n):
                triangle[i][q] += min((triangle[i+1][q],triangle[i+1][q+1])) 
            n-=1
        print(triangle[0][0])

In [31]:
sl = Solution()
sl.minimumTotal([[2],[3,4],[6,5,7],[4,1,8,3]])

11


In [119]:
from itertools import groupby
from operator import itemgetter
class Slide:
    __slots__ = ('tup','len_of','n')
    def __init__(self,start,end):
        self.tup = (start,end)
        self.len_of = (end-start)
        self.n = 1
class Solution:
    def maximalSquare(self, matrix):
        # преобразуем строки в tuple slide, len of slide, 1
        # если по следующей строке slide состоит из 1 по тем же индексам и их длины равны LEN+1 ПРОВЕРКА НА ДОСТИЖЕНИЕ квадрата
        def func(line):
            line = map(int,line)
            temp = groupby(enumerate(line),key=lambda x: x[1]&1)
            temp = filter(itemgetter(0),temp)
            temp = map(itemgetter(1),temp)
            print(list(temp))
            temp = map(next,temp)
            print(list(temp))
            return temp
        return func(matrix[1])


In [120]:
g = [["1","0","1","0","0"],["1","0","1","1","1"],["1","1","1","1","1"],["1","0","0","1","0"]]

In [121]:
sl = Solution()
ms = sl.maximalSquare(g)

[<itertools._grouper object at 0x00000224611531F0>, <itertools._grouper object at 0x0000022461153BB0>]
[]


In [118]:
print(list(ms))

[]


In [ ]:
print(ms[0][